In [3]:
from Crypto.Cipher import DES
import random
import csv

def des_enc(k: bytes, m: bytes) -> bytes:
    """
    Encrypt a message m with a key k using DES as follows:
    c = DES(k, m)

    Note that for DES the key is given as 8 bytes, where the last bit of
    each byte is just a parity bit, giving the actual key of 56 bits, as
    expected for DES. The parity bits are ignored.

    Args:
        k (str): bytestring of length exactly 8 bytes.
        m (str): bytestring containing the message (i.e. a sequence of
            characters such as 'Hello' or '\x02\x04')

    Return:
        bytes: The bytestring ciphertext c
    """
    d = DES.new(k, DES.MODE_ECB)
    c = d.encrypt(m)
    return c


def des_dec(k: bytes, c: bytes) -> bytes:
    """
    Decrypt a message c with a key k using DES as follows:
    m = DES(k, c)

    Note that for DES the key is given as 8 bytes, where the last bit of
    each byte is just a parity bit, giving the actual key of 56 bits, as
    expected for DES. The parity bits are ignored.

    Args:
        k (str): bytestring of length exactly 8 bytes.
        c (str): bytestring containing the ciphertext (i.e. a sequence of
            characters such as 'Hello' or '\x02\x04')

    Return:
        bytes: The bytestring plaintext m
    """
    d = DES.new(k, DES.MODE_ECB)
    m = d.decrypt(c)
    return m

def random_bytes(length):
    result = []
    for _ in range(length):
        result.append(random.randint(0, 255))
    return result

In [4]:
NUM_BYTES_MSG = 8
NUM_BYTES_KEY = 7

N = 10000000
RATIO = 1 / 4

data = []
msgs = set()
keys = set()

for _ in range(N):
    msg = random_bytes(NUM_BYTES_MSG)
    key = random_bytes(NUM_BYTES_KEY)
    
    msg_bytes = bytes(msg)
    key_bytes = bytes(key)
    
    msg_num = int.from_bytes(msg)
    key_num = int.from_bytes(key)
    while msg_num in msgs and key_num in keys:
        msg = random_bytes(NUM_BYTES_MSG)
        key = random_bytes(NUM_BYTES_KEY)
        
        msg_bytes = bytes(msg)
        key_bytes = bytes(key)
        
        msg_num = int.from_bytes(msg)
        key_num = int.from_bytes(key)
    
    msgs.add(msg_num)
    keys.add(key_num)
    
    msg_num = int.from_bytes(msg)
    key_num = int.from_bytes(key)
    
    key.append(0)
    msg_bytes = bytes(msg)
    key_bytes = bytes(key)
    
    
    cipher_bytes = des_enc(key_bytes, msg_bytes)
    cipher = [x for x in cipher_bytes]
    
    
    cipher_num = int.from_bytes(cipher_bytes)
    
    data.append((*msg, *cipher, *key[:NUM_BYTES_KEY]))

file_name = "DES_TRAIN.csv"
delimiter = ','

header = []
for i in range(NUM_BYTES_MSG):
    header.append('M' + str(i))
for i in range(NUM_BYTES_MSG):
    header.append('C' + str(i))
for i in range(NUM_BYTES_MSG):
    header.append('K' + str(i))

# Write the data to the file
with open(file_name, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=delimiter)
    writer.writerow(header)
    writer.writerows(data[:round(RATIO * N)])

# TEST
data_test = data[round(RATIO * N):]
file_name = "DES_TEST.csv"
delimiter = ','

# Write the data to the file
with open(file_name, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=delimiter)
    writer.writerow(header)
    writer.writerows(data_test)